In [ ]:
!pip install --upgrade snowflake-connector-python
!pip install pyarrow==10.0.1

In [ ]:
print('this is an edit')

In [2]:
import requests
import pandas as pd
import time
import snowflake.connector
import glob
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
from datetime import datetime, timedelta

current_date = datetime.now()

# Get the list of months since January 1, 2000
start_date = datetime(2000, 1, 1)
months_since_2000 = [(start_date.year, start_date.month)]
while start_date < current_date:
    start_date = start_date + timedelta(days=32)  # Adding 32 days to make sure we jump to the next month
    start_date = start_date.replace(day=1)  # Setting the day to 1 to get the first day of the month
    months_since_2000.append((start_date.year, start_date.month))
months = months_since_2000[:-1]
months = [f"{year}-{month:02d}" for year, month in months]
months

In [11]:
conn = snowflake.connector.connect(
user='myusername',
password='mypassword',
account='myaccount',
warehouse='AAPL_warehouse',
database='AAPL_database',
schema='firstattempt',
)
cur = conn.cursor()
# username, password, account

In [12]:
cur.execute("CREATE WAREHOUSE IF NOT EXISTS AAPL_warehouse")
cur.execute("CREATE DATABASE IF NOT EXISTS AAPL_database")
cur.execute("CREATE SCHEMA IF NOT EXISTS AAPL_database.firstattempt")
cur.execute("CREATE OR REPLACE STAGE AAPLDATA FILE_FORMAT = (TYPE = 'CSV')")

# Loading AAPL 5 min data from 2000/01/01

The API keys listed below are not the actual API keys

In [ ]:
count= 0
df = pd.DataFrame()
for i in months:
  url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=5min&month={}&outputsize=full&datatype=csv&apikey=myapikey'.format(i)
  data = pd.read_csv(url)
  df = pd.concat([df,data])
  count += 1
  if count % 300 == 0:
    time.sleep(70)
  print(len(df))

In [13]:
df.to_csv('aapl.csv')

In [14]:
cur.execute("""
    PUT file://aapl.csv @AAPLDATA
""")

In [15]:
cur.execute("""CREATE OR REPLACE TABLE AAPL (
        rownum integer,
        TIMESTAMP TIMESTAMP,
        OPEN FLOAT,
        HIGH FLOAT,
        LOW FLOAT,
        CLOSE FLOAT,
        VOLUME INTEGER
)""")

In [16]:
copy_command = """
COPY INTO AAPL
FROM @AAPL_DATABASE.FIRSTATTEMPT.AAPLDATA
FILE_FORMAT = (TYPE = 'CSV' skip_header = 1)
ON_ERROR = 'CONTINUE'
"""
cur.execute(copy_command)

# Load RSI, SMA, MACD

## MA5

In [ ]:
count= 0
df2 = pd.DataFrame()
for i in months:
  url = 'https://www.alphavantage.co/query?function=SMA&symbol=AAPL&interval=5min&time_period=5&month={}&datatype=csv&series_type=open&apikey=myapikey'.format(i)
  data = pd.read_csv(url)
  df2 = pd.concat([df2,data])
  count += 1
  if count % 100 == 0:
    time.sleep(70)
  print(len(df2))

In [61]:
df2

,time,SMA
0,2000-01-31 16:00,0.7706
1,2000-01-31 15:55,0.7656
2,2000-01-31 15:50,0.7616
3,2000-01-31 15:45,0.7584
4,2000-01-31 15:40,0.7566
...,...,...
1335,2023-10-02 04:40,172.5620
1336,2023-10-02 04:35,172.5360
1337,2023-10-02 04:30,172.5820
1338,2023-10-02 04:25,172.6460


In [62]:
df2.to_csv('ma5.csv')

In [63]:
cur.execute("CREATE OR REPLACE STAGE MADATA FILE_FORMAT = (TYPE = 'CSV')")
cur.execute("""
    PUT file://ma5.csv @MADATA
""")

In [64]:
cur.execute("""CREATE TABLE IF NOT EXISTS MA5 (
        rownum integer,
        TIMESTAMP TIMESTAMP,
        SMA FLOAT
)""")

In [65]:
copy_command = """
COPY INTO MA5
FROM @AAPL_DATABASE.FIRSTATTEMPT.MADATA
FILE_FORMAT = (TYPE = 'CSV' skip_header = 1)
ON_ERROR = 'CONTINUE'
"""
cur.execute(copy_command)

## MA 50

In [ ]:
count= 0
df3 = pd.DataFrame()
for i in months:
  url = 'https://www.alphavantage.co/query?function=SMA&symbol=AAPL&interval=5min&time_period=50&month={}&datatype=csv&series_type=open&apikey=myapikey'.format(i)
  data = pd.read_csv(url)
  df3 = pd.concat([df3,data])
  count += 1
  if count % 100 == 0:
    time.sleep(70)
  print(len(df3))

In [68]:
df3.to_csv('ma50.csv')

In [69]:
cur.execute("""
    PUT file://ma50.csv @MADATA
""")
cur.execute("""CREATE TABLE IF NOT EXISTS MA50 (
        rownum integer,
        TIMESTAMP TIMESTAMP,
        SMA FLOAT
)""")

In [70]:
copy_command = """
COPY INTO MA50
FROM @AAPL_DATABASE.FIRSTATTEMPT.MADATA/ma50.csv
FILE_FORMAT = (TYPE = 'CSV' skip_header = 1)
ON_ERROR = 'CONTINUE'
"""
cur.execute(copy_command)

## RSI 14

In [ ]:
count= 0
df4 = pd.DataFrame()
for i in months:
  url = 'https://www.alphavantage.co/query?function=RSI&symbol=AAPL&interval=5min&time_period=14&month={}&datatype=csv&series_type=open&apikey=myapikey'.format(i)
  data = pd.read_csv(url)
  df4 = pd.concat([df4,data])
  count += 1
  if count % 100 == 0:
    time.sleep(70)
  print(len(df4))

In [48]:
df4

,time,RSI
0,2000-01-31 16:00,77.9789
1,2000-01-31 15:55,73.3184
2,2000-01-31 15:50,72.3296
3,2000-01-31 15:45,65.1308
4,2000-01-31 15:40,62.8895
...,...,...
1325,2023-10-02 05:30,44.8452
1326,2023-10-02 05:25,47.2831
1327,2023-10-02 05:20,43.8838
1328,2023-10-02 05:15,52.8497


In [52]:
df4.to_csv('rsi14.csv')

In [53]:
cur.execute("CREATE OR Replace stage RSIDATA FILE_FORMAT = (TYPE = 'CSV')")
cur.execute("""
    PUT file://rsi14.csv @RSIDATA
""")
cur.execute("""CREATE TABLE IF NOT EXISTS RSI14 (
        rownum integer,
        TIMESTAMP TIMESTAMP,
        RSI FLOAT
)""")

In [54]:
copy_command = """
COPY INTO RSI14
FROM @AAPL_DATABASE.FIRSTATTEMPT.RSIDATA
FILE_FORMAT = (TYPE = 'CSV' skip_header = 1)
ON_ERROR = 'CONTINUE'
"""
cur.execute(copy_command)

## MACD

In [ ]:
count= 0
df5 = pd.DataFrame()
for i in months:
  url = 'https://www.alphavantage.co/query?function=MACD&symbol=AAPL&interval=5min&month={}&datatype=csv&series_type=open&apikey=myapikey'.format(i)
  data = pd.read_csv(url)
  df5 = pd.concat([df5,data])
  count += 1
  if count % 300 == 0:
    time.sleep(70)
  print(len(df5))

In [34]:
df5.to_csv('macd.csv')

In [35]:
cur.execute("CREATE STAGE IF NOT EXISTS MACDDATA FILE_FORMAT = (TYPE = 'CSV')")
cur.execute("""
    PUT file://macd.csv @MACDDATA
""")
cur.execute("""CREATE TABLE IF NOT EXISTS MACD (
        rownum integer,
        TIMESTAMP TIMESTAMP,
        MACD FLOAT,
        MACD_hist FLOAT,
        MACD_Signal Float
)""")

In [36]:
copy_command = """
COPY INTO MACD
FROM @AAPL_DATABASE.FIRSTATTEMPT.MACDDATA
FILE_FORMAT = (TYPE = 'CSV' skip_header = 1)
ON_ERROR = 'CONTINUE'
"""
cur.execute(copy_command)

## VWAP

In [ ]:
count= 0
df6 = pd.DataFrame()
for i in months:
  url = 'https://www.alphavantage.co/query?function=VWAP&symbol=AAPL&interval=5min&month={}&datatype=csv&apikey=myapikey'.format(i)
  data = pd.read_csv(url)
  df6 = pd.concat([df6,data])
  count += 1
  if count % 150 == 0:
    time.sleep(70)
  print(len(df6))


In [75]:
df6

,time,VWAP
0,2000-01-31 16:00,0.7421
1,2000-01-31 15:55,0.7421
2,2000-01-31 15:50,0.7411
3,2000-01-31 15:45,0.7404
4,2000-01-31 15:40,0.7399
...,...,...
1339,2023-10-02 04:20,172.5618
1340,2023-10-02 04:15,172.5634
1341,2023-10-02 04:10,172.6172
1342,2023-10-02 04:05,172.5993


In [76]:
df6.to_csv('vwap.csv')

In [77]:
cur.execute("CREATE STAGE IF NOT EXISTS VWAPDATA FILE_FORMAT = (TYPE = 'CSV')")
cur.execute("""
    PUT file://vwap.csv @VWAPDATA
""")
cur.execute("""CREATE TABLE IF NOT EXISTS vwap (
        rownum integer,
        TIMESTAMP TIMESTAMP,
        VWAP Float
)""")

In [81]:
copy_command = """
COPY INTO VWAP
FROM @AAPL_DATABASE.FIRSTATTEMPT.VWAPDATA
FILE_FORMAT = (TYPE = 'CSV' skip_header = 1)
ON_ERROR = 'CONTINUE'
"""
cur.execute(copy_command)

## Still need to load sentiment and government data

# Query Data


In [115]:
cur.execute("""create or replace table AAPLFinal as (
  with A as(select AAPL.*, RSI14.RSI, extract(MONTH FROM TIMESTAMP) Month, EXTRACT(DOW FROM TIMESTAMP) DOW, AAPL.Open - AAPL.Close as Change, AAPL.High - AAPL.Low as movement from AAPL
  left join RSI14
  using(timestamp)
  Order by timestamp),

  B as (
  select A.*, MACD.macd, MACD.macd_hist, MACD.macd_signal from A
  left join MACD
  using(timestamp)
  order by timestamp
  ),

  C as (
    select B.*, MA50.sma ma50 from B
    left join MA50
    using(timestamp)
    order by timestamp
  ),

  D as(
    select C.*, MA5.sma ma5 from C
    left join MA5
    using(timestamp)
    order by timestamp
  ),

  E as (
    select D.*, VWAP.VWAP VWAP from D
    left join VWAP
    using(timestamp)
    order by timestamp
  )
  SELECT * FROM E
  WHERE HOUR(TO_TIMESTAMP(TIMESTAMP)) * 60 + MINUTE(TO_TIMESTAMP(TIMESTAMP)) >= 570
  AND HOUR(TO_TIMESTAMP(TIMESTAMP)) * 60 + MINUTE(TO_TIMESTAMP(TIMESTAMP)) <= 960) """)

In [ ]:
cur.execute("""CREATE or Replace TEMPORARY TABLE temp_table AS
SELECT *, lead(Change) OVER (ORDER BY timestamp) AS Target1
FROM AAPLFinal;""")
query = "SELECT * FROM temp_table where extract(year, timestamp) >= 2001 order by timestamp"
df = pd.read_sql(query, conn)

In [117]:
df

,TIMESTAMP,ROWNUM,OPEN,HIGH,LOW,CLOSE,VOLUME,RSI,MONTH,DOW,CHANGE,MOVEMENT,MACD,MACD_HIST,MACD_SIGNAL,MA50,MA5,VWAP,TARGET1
0,2001-01-02 09:30:00,2266,0.226,0.228,0.225,0.225,11009600,NaN,1,2,0.001,0.003,NaN,NaN,NaN,NaN,0.2254,0.2259,0.000
1,2001-01-02 09:35:00,2265,0.226,0.229,0.223,0.226,5381600,NaN,1,2,0.000,0.006,NaN,NaN,NaN,NaN,0.2258,0.2259,-0.001
2,2001-01-02 09:40:00,2264,0.227,0.229,0.226,0.228,11144000,NaN,1,2,-0.001,0.003,NaN,NaN,NaN,NaN,0.2262,0.2266,0.000
3,2001-01-02 09:45:00,2263,0.228,0.231,0.227,0.228,10628800,NaN,1,2,0.000,0.004,NaN,NaN,NaN,NaN,0.2266,0.2271,0.002
4,2001-01-02 09:50:00,2262,0.228,0.229,0.224,0.226,9900800,NaN,1,2,0.002,0.005,NaN,NaN,NaN,NaN,0.2270,0.2270,0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451792,2023-10-10 15:40:00,51,178.585,178.740,178.540,178.630,515934,52.2100,10,2,-0.045,0.200,-0.0007,0.0802,-0.0810,178.8037,178.6730,178.7896,0.020
451793,2023-10-10 15:45:00,50,178.630,178.680,178.530,178.610,557809,53.5639,10,2,0.020,0.150,0.0088,0.0718,-0.0630,178.7866,178.6870,178.7865,0.380
451794,2023-10-10 15:50:00,49,178.610,178.660,178.120,178.230,1652985,52.8473,10,2,0.380,0.540,0.0146,0.0621,-0.0475,178.7687,178.6690,178.7656,-0.180
451795,2023-10-10 15:55:00,48,178.230,178.450,178.230,178.410,1990496,41.4900,10,2,-0.180,0.220,-0.0114,0.0289,-0.0403,178.7405,178.5570,178.7442,-0.120


# Data Preprocessing